In [169]:
def remove_newlines(serie):
    serie = serie.replace('\n', ' ')
    serie = serie.replace('\\n', ' ')
    serie = serie.replace('\r', ' ')
    serie = serie.replace('\r', ' ')
    serie = serie.replace('    ', ' ')
    serie = serie.replace('   ', ' ')
    serie = serie.replace('  ', ' ')
    serie = serie.replace('-', ' ')
    serie = serie.replace(',', ' ')
    serie = serie.replace('_', ' ')
    serie = serie.replace('📝', '')
    return serie

In [170]:
import requests
import pandas as pd
import json
import os

url = "https://passis-bfd9b877f7d0.herokuapp.com/"

table_list = ['memorias', 'recuperar_lista_documentos']

payload = {}
headers = {}

resultados = []  # Lista para armazenar os valores concatenados de todas as linhas

for table in table_list:
    n_url = url + table
    response = requests.request("GET", n_url, headers=headers, data=payload)
    data = json.loads(response.text)
    df = pd.DataFrame(data)
    # Iterar pelas linhas do DataFrame
    for index, row in df.iterrows():
        concatenated_values = ''
        # Iterar pelas colunas e adicionar o nome da coluna e o valor à string
        for col_name in df.columns:
            concatenated_values += col_name + ': ' + remove_newlines(str(row[col_name])) + '. '
        # Adicionar o nome da tabela, o ID (índice) e o conteúdo concatenado à lista de resultados
        resultados.append([table, index, concatenated_values])

In [171]:
# Criar o DataFrame
df = pd.DataFrame(resultados, columns=['tabela', 'index', 'texto'])

# Agora você tem um DataFrame 'df' com as colunas 'tabela', 'index' e 'texto'
print(df)
df.to_csv('resultados.csv', index=False)
df


                       tabela  index  \
0                    memorias      0   
1                    memorias      1   
2  recuperar_lista_documentos      0   
3  recuperar_lista_documentos      1   

                                               texto  
0  content: perguntas para gartner:   percentual ...  
1  content:  esse é meu primeiro registro de memó...  
2  data_de_upload: 2023 09 10 00:26:46. descricao...  
3  data_de_upload: 2023 09 10 00:31:07. descricao...  


,tabela,index,texto
0,memorias,0,content: perguntas para gartner: percentual ...
1,memorias,1,content: esse é meu primeiro registro de memó...
2,recuperar_lista_documentos,0,data_de_upload: 2023 09 10 00:26:46. descricao...
3,recuperar_lista_documentos,1,data_de_upload: 2023 09 10 00:31:07. descricao...


In [172]:
import tiktoken
# from openai.embeddings_utils import get_embedding, cosine_similarity
tokenizer = tiktoken.get_encoding("cl100k_base")

In [173]:
df['n_tokens'] = df.texto.apply(lambda x: len(tokenizer.encode(x)))
max_tokens = 200


In [174]:
def split_into_many(text, max_tokens = max_tokens):

    # Split the text into sentences
    sentences = text.split('. ')
    print('--------------------', sentences)
    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    print(n_tokens)
    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks

In [175]:
shortened = []

# Loop through the dataframe
for row in df.iterrows():

    # If the text is None, go to the next row
    if row[1]['texto'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the text into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['texto'])

    # Otherwise, add the text to the list of shortened texts
    else:
        shortened.append( row[1]['texto'] )


df2 = pd.DataFrame(shortened, columns = ['texto'])
df2['n_tokens'] = df.texto.apply(lambda x: len(tokenizer.encode(x)))

In [176]:
for i in df2['texto']:
    print (">", i, "\n")

> content: perguntas para gartner:   percentual de empresas investindo em fine tuning   divisão de estratégia entre foundation models  domain models and model hubs.. date_created: 2023 09 23 17:37:47. id: 2.  

> content:  esse é meu primeiro registro de memória. date_created: 2023 09 23 15:22:08. id: 1.  

> data_de_upload: 2023 09 10 00:26:46. descricao: Identidade civil ou Registro Geral Número do documento: MG10192099. id: 5. nome_do_documento: RG. versao: 1.  

> data_de_upload: 2023 09 10 00:31:07. descricao: Passaporte número: GD208356 Data de expedição: 05/11/2021 Validade: 04/11/2031. id: 6. nome_do_documento: Passaporte. versao: 1.  



In [177]:
df

,tabela,index,texto,n_tokens
0,memorias,0,content: perguntas para gartner: percentual ...,58
1,memorias,1,content: esse é meu primeiro registro de memó...,35
2,recuperar_lista_documentos,0,data_de_upload: 2023 09 10 00:26:46. descricao...,55
3,recuperar_lista_documentos,1,data_de_upload: 2023 09 10 00:31:07. descricao...,72


In [178]:
df2

,texto,n_tokens
0,content: perguntas para gartner: percentual ...,58
1,content: esse é meu primeiro registro de memó...,35
2,data_de_upload: 2023 09 10 00:26:46. descricao...,55
3,data_de_upload: 2023 09 10 00:31:07. descricao...,72


In [183]:
import openai

openai.api_key = ('xxxxxxx')

df['embeddings'] = df.texto.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])
df.to_csv('embeddings.csv', index=False)
df.head()

,tabela,index,texto,n_tokens,embeddings
0,memorias,0,content: perguntas para gartner: percentual ...,58,"[0.011579838581383228, -0.0010901413625106215,..."
1,memorias,1,content: esse é meu primeiro registro de memó...,35,"[-0.012386146001517773, -0.0006594319129362702..."
2,recuperar_lista_documentos,0,data_de_upload: 2023 09 10 00:26:46. descricao...,55,"[-0.026302412152290344, 0.011552642099559307, ..."
3,recuperar_lista_documentos,1,data_de_upload: 2023 09 10 00:31:07. descricao...,72,"[-0.022375211119651794, 0.01720244623720646, 0..."


In [185]:
for i in df['n_tokens']:
    print (">", i, "\n")

> 58 

> 35 

> 55 

> 72 

